# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint
from ipywidgets.embed import embed_minimal_html

# Import API key
# from api_keys import g_key

from myconfig import gglkey

# Configure gmaps
gmaps.configure(api_key=gglkey)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Read and display the city weather data
weather_data = pd.read_csv("../output_data/cities_output.csv")
weather_data.head(10)

,City ID,City Name,Country,Cloudiness,Humidity,Latitude,Longitude,Max Temp,Wind Speed,Date
0,0,vaini,TO,90,100,-21.20,-175.20,66.20,4.70,1595197833
1,1,abu dhabi,AE,41,53,24.47,54.37,96.80,6.93,1595197550
2,2,kinablangan,PH,99,78,7.69,126.55,80.49,3.06,1595197834
3,3,clyde river,CA,90,75,70.47,-68.59,46.40,9.17,1595197323
4,4,chara,RU,100,86,56.91,118.26,55.06,0.94,1595197365
5,5,mataura,NZ,88,83,-46.19,168.86,39.00,3.00,1595197286
6,6,barrow,US,40,74,71.29,-156.79,37.40,12.75,1595197834
7,7,pueblo,US,1,17,38.25,-104.61,95.00,5.82,1595197834
8,8,albany,US,53,50,42.60,-73.97,93.00,6.04,1595197692
9,9,vilyuysk,RU,100,91,63.76,121.62,59.90,10.22,1595197755


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
locations = weather_data[['Latitude', 'Longitude']]
humidity = weather_data['Humidity']

# Plot Heatmap
fig = gmaps.figure(zoom_level=3,center=(0,0))

# Create heat layer
heat_layer1 = gmaps.heatmap_layer(locations, weights=humidity,  dissipating=False, max_intensity=90, point_radius=1)

embed_minimal_html('../output_data/Map 1.html', views=[fig])

# Add layer
fig.add_layer(heat_layer1)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
weather_data = weather_data.dropna(axis=0)
temp_df = weather_data.loc[(weather_data["Max Temp"] > 70) & (weather_data["Max Temp"]<80) 
             & (weather_data["Cloudiness"] == 0) & (weather_data["Wind Speed"]<10) & (weather_data["Country"]!="US")]
vacation_df = temp_df.head(10)
vacation_df

,City ID,City Name,Country,Cloudiness,Humidity,Latitude,Longitude,Max Temp,Wind Speed,Date
54,54,abu samrah,SY,0,41,35.30,37.18,74.07,3.87,1595197743
56,56,mantua,IT,0,65,45.17,10.78,72.00,1.99,1595197839
155,155,asfi,MA,0,73,32.30,-9.24,72.88,7.02,1595197848
246,246,nahrin,AF,0,25,36.06,69.13,73.74,2.53,1595197856
302,302,linqing,CN,0,65,36.85,115.71,76.44,7.65,1595197861
317,317,cherskiy,RU,0,46,68.75,161.30,70.34,7.29,1595197740
318,318,jardim,BR,0,44,-21.48,-56.14,74.01,4.59,1595197862
325,325,dubki,RU,0,73,43.02,46.84,75.00,4.47,1595197864
421,421,sungurlu,TR,0,43,40.17,34.37,73.27,5.41,1595197873
428,428,shubarkuduk,KZ,0,44,49.14,56.49,74.91,6.22,1595197761


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
# Crate a new dataframe with the relevant columns for plotting the google map
hotels_df = vacation_df[["City Name", "Country", "Humidity", "Latitude", "Longitude"]]
hotels_df.insert(5, 'Hotel Name', "")
hotels_df

,City Name,Country,Humidity,Latitude,Longitude,Hotel Name
54,abu samrah,SY,41,35.30,37.18,
56,mantua,IT,65,45.17,10.78,
155,asfi,MA,73,32.30,-9.24,
246,nahrin,AF,25,36.06,69.13,
302,linqing,CN,65,36.85,115.71,
317,cherskiy,RU,46,68.75,161.30,
318,jardim,BR,44,-21.48,-56.14,
325,dubki,RU,73,43.02,46.84,
421,sungurlu,TR,43,40.17,34.37,
428,shubarkuduk,KZ,44,49.14,56.49,


In [6]:
# create a params dict that will be updated with new city each iteration
params = {"key" : gglkey,
          "types" : "lodging",
          "radius" : 5000}

# Loop through the cities_pd and run a lat/long search for each city
for index, row in hotels_df.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # Get the latitude & longitude of th city and save it as location paramter
    latitude = row['Latitude']
    longitude = row['Longitude']
    params["location"] = f"{latitude},{longitude}"
    # Retrieve hotel name for the city
    response = requests.get(base_url, params=params).json()
    try:
        # update the name of the current hotel to the current row of the dataframe
        pd.set_option('mode.chained_assignment', None)
        hotels_df.loc[index, "Hotel Name"] = response['results'][0]['name']        
    except Exception as e:
        print(f"Hotel not found in {row['City Name']}")
# Drop rows with hotel name blank
hotels_df.drop(hotels_df[hotels_df['Hotel Name'] == ""].index, inplace = True)
hotels_df

Hotel not found in abu samrah
Hotel not found in nahrin
Hotel not found in shubarkuduk


,City Name,Country,Humidity,Latitude,Longitude,Hotel Name
56,mantua,IT,65,45.17,10.78,Hotel la Favorita
155,asfi,MA,73,32.30,-9.24,Hôtel Abda
302,linqing,CN,65,36.85,115.71,Dongfang Hotel
317,cherskiy,RU,46,68.75,161.30,Gostinitsa
318,jardim,BR,44,-21.48,-56.14,Hotel Estância
325,dubki,RU,73,43.02,46.84,Pansionat Chayka
421,sungurlu,TR,43,40.17,34.37,Sungurlu Ogretmenevi


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City Name}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotels_info = [info_box_template.format(**row) for index, row in hotels_df.iterrows()]
hotellocations = hotels_df[["Latitude", "Longitude"]]

In [8]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(hotellocations, info_box_content=hotels_info)

# Add the layer to the map
fig.add_layer(markers)

# Embed and save the map as an html file
embed_minimal_html('../output_data/Map 2.html', views=[fig])

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))